# Buildings that have ongoing processes

Many buildings have ongoing process that happen over time. This is controlled by an AZ timer function.
```python
@app.function_name(name="factionBuildingTimer")
@app.schedule(schedule="0 */10 * * * *", 
              arg_name="mytimer",
              run_on_startup=RUNNING_LOCALLY)
```



In [3]:
import numpy as np
import pandas as pd

import sys, os, yaml, ssl, asyncio



# mapping to the modules that make the app
sys.path.insert(0, "../..")


In [4]:
ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio

# this is required for running in a Jupyter Notebook.
nest_asyncio.apply()

from app.connectors import cmdb_graph

c = cmdb_graph.CosmosdbClient()

executing local windows deployment


In [71]:
def get_faction_pop_structures(c):
    building_query = f"""
    g.V().has('label','faction').as('faction').in('isIn').has('label','pop').as('pop').out('owns').as('structure').path().by(valueMap())
    """
    c.run_query(building_query)
    faction_res = c.query_to_dict(c.res)
    print(f"EXOADMIN: number of items: {len(faction_res)}")
    for item in faction_res:
        item['action'] = 'structure'
    return faction_res

faction_res = get_faction_pop_structures(c)

EXOADMIN: number of items: 5


Here we are attaching `structure` to the message so that the EventHub trigger can catch it and process it. Other than that, it does nothing. 

In [72]:
message = faction_res[0]
message['action']

'structure'

In [73]:
message.keys()

dict_keys(['faction', 'pop', 'structure', 'action'])

In [74]:
message['structure']

{'name': 'Oil Well',
 'objid': '5461385315509',
 'ownedBy': '1852583025275',
 'type': 'oil_well',
 'description': 'Produces energy by extracting hydrocarbons from the ground.',
 'owned_by': 'pop',
 'faction_augments': '{wealth: 1}',
 'requires_attr': '{industry: 0.3}',
 'renews_location_resource': '{hydrocarbons: 10}',
 'effort': '2',
 'render_type': 'mesh',
 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
 'objtype': 'building',
 'id': '5461385315509'}

In [75]:
message['pop']

{'isIdle': 'true',
 'name': 'An To',
 'objid': '1852583025275',
 'conformity': 0.46,
 'literacy': 0.492,
 'aggression': 0.482,
 'constitution': 0.482,
 'health': 0.7,
 'isIn': '2265012067295',
 'industry': 0.482,
 'wealth': 0.487,
 'factionLoyalty': 0.286,
 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
 'objtype': 'pop',
 'id': '1852583025275'}

In [76]:
message['faction']

{'wealth': 3,
 'name': 'An',
 'objid': '2265012067295',
 'lat': -0.117,
 'long': -0.0,
 'infrastructure': 0,
 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
 'objtype': 'faction',
 'id': '2265012067295'}

## faction_augments

In [80]:
def augemt_faction(c, message):
    faction = message['faction']
    augment = yaml.safe_load(message['structure']['faction_augments'])
    for item in augment.keys():
        old_value = faction.get(item,0)
        new_value = old_value + augment[item]
        print("EXOADMIN: ", item, "has changed from", old_value, "to" ,new_value)
        augment_query = f"""
            g.V().has('label','faction').has('objid','{faction["objid"]}').property('{item}', {new_value})
        """
        c.run_query(augment_query)
        print(augment_query)
    return augment  

augemt_faction(c, message)

EXOADMIN:  wealth has changed from 3 to 4

            g.V().has('label','faction').has('objid','2265012067295').property('wealth', 4)
        


{'wealth': 1}

In [81]:
fac_id = message['faction']['objid']
check_faction = f"""
        g.V().has('label','faction').has('objid','{fac_id}').valueMap()
    """
c.run_query(check_faction)
c.res

[{'wealth': [4],
  'name': ['An'],
  'objid': ['2265012067295'],
  'lat': [-0.117],
  'long': [-0.0],
  'infrastructure': [0],
  'userguid': ['8d5b667f-b225-4641-b499-73b77558ff86'],
  'objtype': ['faction']}]